In [ ]:
!pip install -r requirements.txt

## Import the package 

We start by importing the `siibra` package and other necessary libraries.

In [ ]:
import siibra
from siibra import explorer
import matplotlib.pyplot as plt
from nilearn import plotting

## I. Find and display maps

### 1. Browse available parcellations

In [ ]:
siibra.parcellations.dataframe

In [ ]:
for parc in siibra.parcellations:
    print(parc.name)

### 2. Select Von Economo & Koskinas and display the region hierarchy

In [ ]:
voneconomo = siibra.parcellations["von economo"]
voneconomo.name

In [ ]:
voneconomo.render_tree()

In [ ]:
voneconomo.find("granularis")

In [ ]:
voneconomo.LICENSE

In [ ]:
voneconomo.publications

### 3. Get the labelled map of von economo koskinas

In [ ]:
voneconomo_map = voneconomo.get_map("MNI ICBM 2009c nonlinear asym")
img = voneconomo_map.fetch()
print(type(img))

### 4. Plot the map on MNI ICBM 2009c nonlinear asym

In [ ]:
plotting.view_img(
    img,
    cmap=voneconomo_map.get_colormap(),
    symmetric_cmap=False,
    resampling_interpolation="nearest"
)

### HANDS ON

Get the map of julich brain 3.1 and its map, then plot it. 

In [ ]:
# Get julich brain 2.9 parcellation


In [ ]:
# Get the corresponding map in the reference space MNI ICBM 2009c nonlinear asym


In [ ]:
# Plot the map


## II. Using siibra-python to characterize motor and language areas in the Julich-Brain cytoarchitectonic atlas 

### 1. Select Julich-Brain 3.1 parcellation

In [ ]:
# access Julich-Brain in version 3.1
julichbrain = siibra.parcellations.get("julich brain 3.1")
print(julichbrain.render_tree())

### 3. Select precentral gyrus from Julich-Brain

In [ ]:
# select 4p left from julich brain
region = julichbrain.get_region("precentral gyrus")
print("object type:", type(region))
print("name:", region)
print("parcellation:", region.parcellation)

### HANDS ON: render tree of the "precentral gyrus" region

### 4. Select two children of precentral gyrus, specifcally 4p left and 44 left

In [ ]:
regions = [
    julichbrain.get_region("4p left"),
    julichbrain.get_region("44 left")
]
regions

### 5. Plot the probability maps of these regions

In [ ]:
# fetch and display their probability maps
for region in regions:
    pmap = region.get_regional_map(
        space=siibra.spaces["MNI ICBM 2009c nonlinear asym"],
        maptype="statistical"
    )
    plotting.plot_stat_map(pmap.fetch(), title=region.name, cmap="magma")

### 6. Get receptor density profiles for one of the regions

In [ ]:
print(regions[0])
receptor_profiles = siibra.features.get(regions[0], "receptor density profile")[0]
print(receptor_profiles.name)
print("Found profiles are anchored at:", receptor_profiles.anchor)

In [ ]:
print(receptor_profiles.description)

In [ ]:
for rp in receptor_profiles:
    print(rp.name)

### 7. Plot GABAB receptor density distributions for both regions

In [ ]:
receptor = "GABAB"

In [ ]:
for i, region in enumerate(regions):
    # query for receptor profiles
    receptor_profiles = siibra.features.get(
        region, "receptor density profile"
    )
    print("number of receptor density profiles found:", len(receptor_profiles[0]))
    # get one with GABAB type and plot it
    receptor_profiles[0].get_element(receptor).plot(label=region.name)
plt.legend()

### HANDS ON: Plot AMPA receptor density distributions for both regions

### 8. Run a differential gene expression analysis for the two regions

In [ ]:
# setup and run the analysis
genes = ['GABBR1', 'GABBR2']

geneexps = {}
for region in regions:
    geneexps[region] = siibra.features.get(region, "GeneExpression", gene=genes)[0]
    geneexps[region].plot()

### 9. Plot the extracted sample positions of the microarray data

In [ ]:
geneexps[regions[0]].anchor.location

In [ ]:
geneexps[regions[0]].anchor.location.coordinates

In [ ]:
print(geneexps[regions[0]].anchor.location.space)

In [ ]:
for region in regions:
    pmap = region.get_regional_map(
        siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
        siibra.MapType.STATISTICAL
    )
    display = plotting.plot_glass_brain(pmap.fetch(), cmap="viridis", title=region.name)
    display.add_markers(geneexps[region].anchor.location.coordinates, marker_size=5)

### 10. Plot the coordinates on BigBrain

In [ ]:
# get BigBrain space
bigbrain = siibra.spaces["bigbrain"]

In [ ]:
# warp the points to BigBrain space
points_in_mni152 = geneexps[regions[0]].anchor.location
points_in_bigbrain = points_in_mni152.warp(bigbrain)
print(points_in_bigbrain.space)
print(points_in_bigbrain.coordinates)

### 11. Plot on BigBrain template

In [ ]:
bigbrain_template = bigbrain.get_template()
display = plotting.plot_img(bigbrain_template.fetch(), cmap="grey", cut_coords=points_in_bigbrain.centroid.coordinate)
display.add_markers(points_in_bigbrain.coordinates, marker_size=5)

## III. Download high-resolution BigBrain chunks

### HANDS ON
You can query for high resolution sections using the regions as well. In this case, can you query for 4p left?

In [ ]:
# No idea what the modality is? Try:
# siibra.features.render_ascii_tree("Feature")

### 1. While we could use the above query, we can also use an explorer link to obtain the volume of interest, aka bounding box

In [ ]:
url = "https://atlases.ebrains.eu/viewer/#/a:juelich:iav:atlas:v1.0.0:1/t:minds:core:referencespace:v1.0.0:a1655b99-82f1-420f-a3c2-fe80fd4c8588/p:minds:core:parcellationatlas:v1.0.0:94c1125b-b87e-45e4-901c-00daee7f2579-bb/@:0.0.0.-W000.._eCwg.2-FUe3._-s_W.2_evlu..7LIy..BSR0.2_l20~.Qi-0..9I-/vs:v2-ff011b0b"
voi = explorer.decode_url(url).bounding_box
print(voi)

### 2.  Plot the BigBrain template for the view port of the above url

In [ ]:
chunk = bigbrain_template.fetch(voi=voi, resolution_mm=0.2)
plotting.view_img(chunk, bg_img=None, cmap='gray', vmin=0, vmax=255, symmetric_cmap=False)

### 3. Zoom in 10 percent and query for sections overlapping with this bounding box

In [ ]:
voi_zoomed = voi.zoom(0.1)
sections = siibra.features.get(voi_zoomed, "CellbodyStainedSection")
for section in sections:
    print(section.name)

### 4. Fetch and plot chunk of section 1049

In [ ]:
section = sections[2]
print(section)

In [ ]:
section_chunk = section.fetch(voi=voi_zoomed.intersection(section), resolution_mm=-1)
fig = plt.figure(figsize=(10,10))
plotting.plot_img(section_chunk, bg_img=None, cmap='gray', display_mode='y', figure=fig)